In [6]:
import sde
import networkx as nx
import random
import portion

# Lessam

In [7]:
# incomplete data - mix of observed and hidden nodes
def addIncompleteNodes(gm, num_nodes):
    node_list = []
    observed_nodes = int(num_nodes/2)  # half of the total nodes are observed
    hidden_nodes = num_nodes - observed_nodes # the remaining nodes are hidden
    
    for i in range(observed_nodes):
        random_prob = random.random()
        #nx.set_node_attributes(gm, random_prob, 'weight')
        
        gm.add_node(i, weight=random_prob)
        
    for i in range(hidden_nodes):
        gm.add_node(i)

    return gm

In [8]:
def makeIncompleteDirectedGraph(num_nodes, num_edges):
    dg = nx.DiGraph() # creates directed graph
    dg = addIncompleteNodes(dg, num_nodes)
    dg = addEdges(dg, num_nodes, num_edges)
    return dg

In [9]:
#updated to program expansion when a hidden node is a vacuous messsage
# Update2: returns a subgraph rather than list of nodes
# Update3: if a hidden note is a root node, a random edge is directed from a random observed node to the hidden node
def getSubset(dg, observed_node):
    """accepts a graph and node name, returns a subgraph of that node's neighbors and non-d-separated nodes
        (includes the original node also)"""
    #get all neighbors
    children = list(dg.successors(observed_node))
    parents = list(dg.predecessors(observed_node))
    
    #get all non-d-separated nodes
    non_d_separated = []
    for n in list(dg.nodes()):
        n_children = list(dg.predecessors(n))
        commonDescendants = list(set(children) & set(n_children))
        if commonDescendants:
            non_d_separated += commonDescendants
            
    node_subset = [observed_node] + children + parents + commonDescendants
    
    # check for vacuous nodes, add an edge/parent if exists
    hidden_nodes = getHiddenNodes(dg)
    subset_hidden_nodes = list(set(node_subset) & set(hidden_nodes))
    for h in subset_hidden_nodes:
        parents_shn = list(dg.predecessors(h))
        if not parents_shn: 
            obsNodes = getObservedNodes(dg)
            randParent = random.choice(obsNodes)
            dg.add_edge(randParent, h, weight= random.uniform(0,1))
            parents_shn = [randParent]
        if not list(set(parents_shn) & set(node_subset)):
            node_subset.append(parents_shn[0])
            
    subgraph = dg.subgraph(node_subset) #generate the subgraph
    return subgraph

In [10]:
# find all hidden (i.e. unweighted) nodes
def getHiddenNodes(dg):
    """accepts graph, returns list of nodes with no attached weight"""
    hidden_nodes = []
    dict_weights = nx.get_node_attributes(dg, 'weight')
    obsNodes = list(dict_weights.keys())
    
    for n in list(dg.nodes()):
        if (n not in obsNodes):
            hidden_nodes.append(n)
            
    return hidden_nodes

In [11]:
#find all observed (weighted) nodes in a graph
def getObservedNodes(dg):
    """accepts a graph, returns a list of the observed/weighted nodes in the graph"""
    weights_dict = nx.get_node_attributes(dg, 'weight')
    observed_nodes = list(weights_dict.keys())
    return observed_nodes

Marginal probability interval

In [12]:
# Updated to account for hidden nodes with no observed ancestors
def minMargInterval(dg, subdg, hidden_node):
    """accepts a graph and specified unobserved node, returns the lower bound for that node's marginal likelihood interval"""
    ancestors = list(dg.predecessors(hidden_node))
    observed_nodes = getObservedNodes(subdg)
    observed_ancestors = list(set(ancestors) & set(observed_nodes))
    
    if not observed_ancestors:
        descendants = list(dg.successors(hidden_node))
        if not list(set(descendants) & set(observed_nodes)):
            minMarg = .5
        else:
            maxMarg = maxMargInterval(dg, subdg, hidden_node)
            minMarg = 1-maxMarg
    else:
        ancList = []
        cpList = []
        for n in observed_ancestors:
            n_weight_dict = nx.get_node_attributes(dg, 'weight')
            n_weight = n_weight_dict.get(n)
            ancList.append(n_weight)

            e_weight_dict = nx.get_edge_attributes(dg, 'weight')
            edge_weight = e_weight_dict.get((n, hidden_node))
            cpList.append(edge_weight)

            minMarg = numpy.prod(cpList)
        return minMarg

In [13]:
# Updated to account for hidden nodes with no observed descendaants 
def maxMargInterval(dg, subdg, hidden_node):
    descendants = list(dg.successors(hidden_node))
    observed_nodes = getObservedNodes(subdg)
    observed_descendants = list(set(descendants) & set(observed_nodes))
    
    if not observed_descendants:
        ancestors = list(dg.successors(hidden_node))
        if not list(set(ancestors) & set(observed_nodes)):
            maxMarg = .5
        else:
            minMarg = minMargInterval(dg, subdg, hidden_node)
            maxMarg = 1-minMarg
            
    else:
        descList = []
        cpList = []
        for n in observed_descendants:
            n_weight_dict = nx.get_node_attributes(dg, 'weight')
            n_weight = n_weight_dict.get(n)
            descList.append(n_weight)

            e_weight_dict = nx.get_edge_attributes(dg, 'weight')
            edge_weight = e_weight_dict.get((hidden_node, n))
            cpList.append(edge_weight)

        maxMarg = numpy.prod(cpList)
    return maxMarg

"""Issue: maxMarg and minMarg sometimes return None bounds. This also disrupts LessamInterval."""

'Issue: maxMarg and minMarg sometimes return None bounds. This also disrupts LessamInterval.'

In [14]:
# returns the interval-valued marginal probability of a hidden node 
def LessamInterval(dg, subdg, hidden_node):
    lower_bound = minMargInterval(dg, subdg, hidden_node)
    upper_bound = maxMargInterval(dg, subdg, hidden_node)
    marg_prob_interval = portion.closed(lower_bound, upper_bound)
    return marg_prob_interval

